## Assignment 2

In [1]:
#Import required libraries
import cv2
import numpy as np

# Provided fundamental matrix relating im1 and im3
Fmatrix13 = np.array([
    [6.04444988555117e-08, 2.5676214074219e-07, -0.000602529673152695],
    [2.45555247713476e-07, -8.38881173687142e-08, 0.000759823033686890],
    [-0.00044444369704832, 0.000390321707113558, 0.99999361609429]
])

# Provided fundamental matrix relating im2 and im3
Fmatrix23 = np.array([
    [3.0399452899916e-08, 2.65672654114295e-07, -0.0007870550254997210],
    [4.67660901933558e-08, -1.11709498607908e-07, -0.00169128012255720],
    [-1.36301618255506e-06, 0.00140690091935593, 0.99999271101569]
])

### First Question
You are required to write a code that loads the image pair 2-3 and use the provided fundamental matrix (Fmatrix23) to compute and display the corresponding epipolar lines for manually selected points. (i.e. select manually 5 points in im2 and display the corresponding epipolar lines (with different line colors)on im3. Validate visually that the epipolar lines go through the selected points in the other image.)

In [2]:
# Load the images with cv2 library
im2 = cv2.imread('florence2.jpg')
im3 = cv2.imread('florence3.jpg')

# Function to select points with mouse clicks on the image
def select_points(img, points):
    def mouse_click(event, x, y, flags, param): # Function to handle mouse click events
        if event == cv2.EVENT_LBUTTONDOWN and len(points) < 5:# Check for left mouse button click and limit the number of points to 5
            points.append((x, y))
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)# Draw a green circle at the clicked point for visual feedback
    
    # Create a window named 'Select Points' to display the image
    cv2.namedWindow('Select Points', cv2.WINDOW_NORMAL)# To view images in normal size
    cv2.imshow('Select Points', img)
    cv2.setMouseCallback('Select Points', mouse_click)# Set mouse callback function to handle mouse events

    # Continue to display the image until 5 points are selected or 'q' is pressed
    while len(points) < 5:
        cv2.imshow('Select Points', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):# Check for 'q' key press to exit the loop
            break
    cv2.destroyAllWindows()

# Function to compute the epipolar lines for the selected points from images
def compute_epipolar_lines(points, Fmatrix):
    lines = []
    for pt in points:
        #Convert point to homogeneous coordinates,'point' should be in the homogeneous coordinates[x, y, 1]
        pt_homogeneous = np.array([pt[0], pt[1], 1])
        # Compute the epipolar line using the fundamental matrix
        line = np.dot(Fmatrix, pt_homogeneous)
        lines.append(line)
    return np.array(lines)

# Function to draw epipolar lines on the image
def draw_epipolar_lines(img, lines):
    for line in lines:
        color = tuple(np.random.randint(0, 255, 3).tolist())# Assign a random color to each epipolar line
        a, b, c = line
        # Find two points on the epipolar line to draw the line
        x0, y0 = map(int, [-c / a, 0])  # x0 = -c/a, y0 = 0 , Intersection with y-axis (x=0)
        x1, y1 = map(int, [0, -c / b])  # x1 = 0, y1 = -c/b , Intersection with x-axis (y=0)
        image = cv2.line(img, (x0, y0), (x1, y1), color, 2)
    return img

# Main process
points_im2 = []  # Initialize list to store points from im2
select_points(im2.copy(), points_im2)  # Select points on im2
epilines_im3 = compute_epipolar_lines(points_im2, Fmatrix23)  # Compute epipolar lines for im3
im3_with_epilines = draw_epipolar_lines(im3.copy(), epilines_im3)  # Draw epipolar lines on im3

# Display the image with epipolar lines in a window
cv2.namedWindow('Epipolar Lines on im3', cv2.WINDOW_NORMAL)# Display the image with epipolar lines in a resizable window
cv2.imshow('Epipolar Lines on im3', im3_with_epilines)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Second Question
Manually select 5 corresponding points in image pair 1-2, and find a way to compute the projection of these points on the third image. (i.e. select 5 points in im1 and find and select the same point locations in im2. Transfer these points on im3 using epipolar geometry.)

In [4]:
# Load the images
im1 = cv2.imread('florence1.jpg')
im2 = cv2.imread('florence2.jpg')
im3 = cv2.imread('florence3.jpg')

points_im1 = []  # Initialize list to store points from im2
select_points(im1.copy(), points_im1)  # Select points on im2
points_im2 = []  # Initialize list to store points from im2
select_points(im2.copy(), points_im2)  # Select points on im2

epilines_im3 = compute_epipolar_lines(points_im2, Fmatrix23)  # Compute epilines for im3
im3_with_epilines = draw_epipolar_lines(im3.copy(), epilines_im3)  # Draw epilines on im3


def line_intersection(line1, line2):
    """
    Find the intersection of two lines given in homogeneous coordinates.
    """
    x, y, z = np.cross(line1, line2)
    if z == 0:  # If lines are parallel or coincident
        return None
    return (x/z, y/z)

def find_corresponding_points_in_im3(points_im1, points_im2, Fmatrix13, Fmatrix23):
    """
    Find corresponding points in image 3 (im3) using epipolar geometry.
    """
    # Compute epipolar lines for points in image 1 and image 2
    epipolar_lines_im1 = compute_epipolar_lines(points_im1, Fmatrix13)
    epipolar_lines_im2 = compute_epipolar_lines(points_im2, Fmatrix23)
    
    corresponding_points_im3 = []
    # Iterate through corresponding epipolar lines in image 1 and image 2
    for line1, line2 in zip(epipolar_lines_im1, epipolar_lines_im2):
        point = line_intersection(line1, line2)
         # Calculate the intersection point of the epipolar lines
        if point is not None: # If intersection point exists
            corresponding_points_im3.append(point)
    return corresponding_points_im3

# Find the corresponding points in im3 using epipolar geometry
corresponding_points_im3 = find_corresponding_points_in_im3(points_im1, points_im2, Fmatrix13, Fmatrix23)

# Mark the corresponding points in im3_with_epilines
for point in corresponding_points_im3:
    if point is not None:
        # Mark the corresponding points in red
        cv2.circle(im3_with_epilines, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)


# Display the updated image with epilines and projected points on im3
cv2.namedWindow('Image 3 with epipolar lines and corresponding points', cv2.WINDOW_NORMAL)
cv2.imshow('Image 3 with epipolar lines and corresponding points', im3_with_epilines)

cv2.resizeWindow('Image 3 with epipolar lines and corresponding points', im3_with_epilines.shape[1], im3_with_epilines.shape[0])  

cv2.waitKey(0)
cv2.destroyAllWindows()

### Third Question
Can you compute an accuracy for the projected points on im3 in Question 2? Comment on possible reasons for projection errors?

To compute the accuracy of the projected points onto image 3 (florence3.jpg), we typically need a set of known, accurate points to compare against the projected points. These known points, known as 'ground truth,' are manually annotated or obtained from reliable sources, representing the true locations where the points from image 1 (florence1) and image 2 (florence2) should appear on image 3.  However, the image provided does not contain this ground truth data, so we can't compute accuracy in the traditional sense.

Nevertheless, if we had such ground truth points, we could compute the accuracy by measuring the Euclidean distance between the projected points and the ground truth points. Smaller distances indicate higher accuracy—meaning the projected points closely align with the expected true positions. This comparison assists in evaluating how accurately the projection aligns with the intended positions in image 3, providing insight into the alignment quality of the projected points with their expected locations.

The projection errors may arise due to various reasons:

- Calibration Issues: Inaccurate camera calibration parameters or inconsistencies in the camera model used for the computation of fundamental matrices can cause projection inaccuracies.

- Image deformation: If the images have undergone significant transformations, such as rotation, scaling, or non-planar motion, the computed projection may not accurately reflect the corresponding points in the third image.

- Inaccuracies in the fundamental matrix: The fundamental matrix is estimated from the images, and any inaccuracies in the estimation can lead to projection errors.

- Incorrect Correspondences: If the selected corresponding points in images 1 and 2 are not accurately matched, it can lead to errors in computing the epipolar lines, resulting in inaccurate projection onto image 3.

Also reasons like missing depth information, algorithm limitations and image deformation can cause projection error.